# 6.3) Supervised Euclidean
A U-Net is trained here to predict a displacement field and the euclidean distance is taken as error.

# Import statements
Following packages are necessary:

In [1]:
import random
import utils
import os
import imageio
import json
from simplegen import SIMPLESequence

import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow.keras.callbacks import CSVLogger

from unet import UNet
from unet import preprocess_input as pre_une

from matplotlib import pyplot as plt

# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.get_visible_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Data
Import training images

In [4]:
store_path = 'experiments/6_3_euclidean'

path_fixed = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train/0_f.json'
path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train'
path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/validation'
path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/test'

width = 224
height = 224

# Seeding
Seeds are set to ensure reproducible training.

In [5]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [6]:
model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 224, 224, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
A batch with only one image is used.

# Training Preparation
Prepare the settings for training the model.

In [7]:
image_ids = None

train_gen = SIMPLESequence(path_train, path_fixed,
                           batch_size=4,
                           image_ids=image_ids,
                           preprocess_input=pre_une,
                           shuffle=True)

validation_gen = SIMPLESequence(path_validation, path_fixed,
                                batch_size=4,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

In [ ]:
logger = CSVLogger(store_path + "/log")

callbacks = [logger]

model.compile(optimizer='adam', loss = utils.msed_loss)
#model.fit_generator(train_gen,
#          validation_data=validation_gen,
#          callbacks=callbacks,
#          epochs=100)


model.fit(train_gen,
          epochs=300,
          callbacks=callbacks,
          validation_data=validation_gen,
          validation_freq=1,
          #use_multiprocessing=True,
          workers=10,
          max_queue_size=32)

Epoch 1/300
(None, None, None, None)
> /home/julian/Documents/Studium/MT-Masterarbeit/Workspace/endolas/utils.py(124)_custom_loss()
-> for batch_index in range(0, batch_size):
(Pdb) labels
<tf.Tensor 'IteratorGetNext:1' shape=(None, None, None, None) dtype=float32>
(Pdb) ux = prediction[1,:,:,0]
(Pdb) ux.shape
TensorShape([224, 224])
(Pdb) ux = prediction[0,:,:,0]
(Pdb) ux = prediction[7,:,:,0]
(Pdb) ux = prediction[8,:,:,0]
(Pdb) ux
<tf.Tensor 'msed_loss/strided_slice_3:0' shape=(224, 224) dtype=float32>
(Pdb) ux.shape
TensorShape([224, 224])
(Pdb) prediction
<tf.Tensor 'model/segmentation/Conv2D:0' shape=(None, 224, 224, 2) dtype=float32>
(Pdb) kb.batch_get_value(labels)
*** NameError: name 'kb' is not defined
(Pdb) keras.backend.batch_get_value(labels)
*** RuntimeError: Cannot get value inside Tensorflow graph function.
(Pdb) keras.backend.batch_flatten(labels)
<tf.Tensor 'msed_loss/Reshape:0' shape=(None, None) dtype=float32>
(Pdb) keras.backend.batch_flatten(prediction)
<tf.Tensor

In [ ]:
model.save_weights(store_path + "/weights.hdf5")

In [ ]:
X, y = validation_gen[1]

y_pred = model.predict(X)

u_x = y_pred[0,:,:,0]
u_y = y_pred[0,:,:,1]

plt.imshow(y_pred[0,:,:,0], cmap="gray")

plt.imsave(store_path + "/u_x.png", u_x, cmap="gray")
plt.imsave(store_path + "/u_y.png", u_y, cmap="gray")

u_x.dump(store_path + "/u_x")
u_y.dump(store_path + "/u_y")

In [ ]:
def plot_cube(img, x, y, val):
    
    img[y][x] = val
    img[y][x-1] = val
    img[y][x+1] = val
    img[y-1][x] = val
    img[y-1][x-1] = val
    img[y-1][x+1] = val
    img[y+1][x] = val
    img[y+1][x-1] = val
    img[y+1][x+1] = val  

In [ ]:
y[0,0,0,0]

In [ ]:
warp = np.zeros((width, height))

for index in range(0,25):
    x_pos = int(y[0, index, 0, 0])
    y_pos = int(y[0, index, 1, 0])
    
    plot_cube(warp, x_pos, y_pos, 1) #blue
    
    ux_field = y_pred[0,:,:,0]
    uy_field = y_pred[0,:,:,1]
    
    ux = ux_field[y_pos][x_pos]
    uy = uy_field[y_pos][x_pos]
    
    x_pos = int(round(x_pos + ux))
    y_pos = int(round(y_pos + uy))
            
    plot_cube(warp, x_pos, y_pos, 2) #green    
    
    x_pos = int(y[0, index, 0, 1])
    y_pos = int(y[0, index, 1, 1])
    
    plot_cube(warp, x_pos, y_pos, 3) #yellow    
    
plt.imshow(warp)
plt.imsave(store_path + "/warp.png", warp)